In [ ]:
from math import sqrt

# Calculo do erro

In [ ]:
def erro_absoluto(referencia, estimativa):
    return abs(referencia - estimativa)


def erro_relativo(referencia, estimativa):
    return abs((referencia, estimativa) / estimativa)


# Derivação numerica

In [ ]:
DELTA_DERIVACAO_PADRAO = 10**-4


def derivada_direita(f, delta=DELTA_DERIVACAO_PADRAO):
    return lambda x: (f(x + delta) - f(x)) / delta


def derivada_esquerda(f, delta=DELTA_DERIVACAO_PADRAO):
    return lambda x: (f(x - delta) - f(x)) / delta


def derivada_central(f, delta=DELTA_DERIVACAO_PADRAO):
    return lambda x: (f(x + delta) - f(x - delta)) / (2 * delta)


# Funções de parada

In [ ]:
def limite_iteracao(i):
    return lambda est: est.i >= i


def parada_erro_absoluto(referencia, e):
    return lambda est: erro_absoluto(referencia, est.x) < e


def parada_erro_relativo(referencia, e):
    return lambda est: erro_relativo(referencia, est.x) < e


def parada_da_bissecao(e):
    return lambda est: est.b - est.a < 2 * e


def parada_da_falsa_posicao(e):
    return lambda est: est.f(est.x - e) * est.f(est.x + e) < 0


# Método da bisseçao

In [ ]:
class EstatisticaAB:
    def __init__(self, i, f, a, b, x, y) -> None:
        self.i = i
        self.f = f
        self.a = a
        self.b = b
        self.x = x
        self.y = y

    def __str__(self) -> str:
        return f"iteracao: {self.i}, a: {self.a}, b: {self.b}, x: {self.x}, y: {self.y}"


In [ ]:
def bissecao(a, b, f, condicao_parada, observador=lambda _: None):
    if a > b:
        a, b = b, a
    i = 1
    x = (a + b) / 2
    y = f(x)
    est = EstatisticaAB(i, f, a, b, x, y)
    observador(est)
    while not condicao_parada(est):
        i += 1
        if f(a) * y < 0:
            b = x
        else:
            a = x
        x = (a + b) / 2
        y = f(x)
        est_erro = (b - a) / 2
        est = EstatisticaAB(i, f, a, b, x, y)
        observador(est)
    return est


# Método da falsa posição

In [ ]:
def falsa_posicao(a, b, f, condicao_parada, observador=lambda _: None):
    if a > b:
        a, b = b, a
    i = 1
    x = (a * f(b) - b * f(a)) / (f(b) - f(a))
    y = f(x)
    est = EstatisticaAB(i, f, a, b, x, y)
    observador(est)
    while not condicao_parada(est):
        i += 1
        if f(a) * y < 0:
            b = x
        else:
            a = x
        x = (a * f(b) - b * f(a)) / (f(b) - f(a))
        y = f(x)
        est_erro = (b - a) / 2
        est = EstatisticaAB(i, f, a, b, x, y)
        print(est)
    return est


# Método do ponto fixo

In [ ]:
class EstatisticaX:
    def __init__(self, i, f, fi, x, y) -> None:
        self.i = i
        self.f = f
        self.fi = fi
        self.x = x
        self.y = y

    def __str__(self) -> str:
        return f"iteracao: {self.i}, x: {self.x}, y:{self.y}"


In [ ]:
def ponto_fixo(x, f, fi, condicao_parada, observador=lambda _: None):
    i = 1
    y = f(x)
    est = EstatisticaX(i, f, fi, x, y)
    observador(est)
    while not condicao_parada(est):
        i += 1
        x = fi(x)
        y = f(x)
        est = EstatisticaX(i, f, fi, x, y)
        observador(est)
    return est


# Método de Newtown-Raphson

In [ ]:
def newton_raphson(x, f, condicao_parada, observador=lambda x: None):
    d = derivada_central(f)
    fi = lambda x: x - f(x) / d(x)
    return  ponto_fixo(x, f, fi, condicao_parada, observador)

# Amostras

In [ ]:
def amostra_linear(x):
    return 5 * x + 2

def amostra_poli(x):
    return x ** 2 + x - 6

def amostra_poli_iter(x):
    return sqrt(6 - x) 


# Playground 

In [ ]:
resultado = newton_raphson(1.5, amostra_poli, limite_iteracao(10), print)
print(resultado)
